# Glonas Position

What are the difference between GPS and Glonass (necessary) for calculate the position.

### References ###
[The Receiver Independent Exchange Format](ftp://igs.org/pub/data/format/rinex303.pdf) -
[GLONASS ICD 5.1](https://www.unavco.org/help/glossary/docs/ICD_GLONASS_5.1_(2008)_en.pdf) - 
[The GPSTk: GLONASS, RINEX Version 3.00 and More](http://www.gpstk.org/foswiki/pub/Documentation/GPSTkPublications/gpstk-ion-gnss-2009.pdf) - 
[GPS + GLONASS combined](https://fenix.tecnico.ulisboa.pt/downloadFile/395145496449/thesis.pdf) - [RINEX description](https://igscb.jpl.nasa.gov/igscb/data/format/rinex211.txt)


In [178]:
import numpy as np
import gpstk
import matplotlib.pyplot as plt
from IPython.display import display, HTML 
%matplotlib inline

Comparation of Constelations
![Digit of the RINEX](img/comparation.png)

[step-step for a Glonass position](http://gage14.upc.es/gLAB/HTML/GNSS_standard_format_files.pdf)


 <font color='red'>**[1]**</font>  **How do you identify this file as a GLONASS Navigation message?***
 <font color='red'>**RTA**</font>→ The RINEX File Type value is a ’G’

Se debe incluir un archivo adicional con terminación en "g" que es el Navigator Data para Glonass

![Digit of the RINEX](img/digitRINEX.png)

In [179]:
navfile = "glonass/badg3170.15n" #"data/bogt0010.15n" #aira0690
obsfile = "glonass/badg3170.15o" #"data/bogt0010.15o"
glofile = "glonass/badg3170.15g" #"data/bogt0010.15o"

In [291]:
!head -29 $glofile

 <font color='red'>**[2]**</font> **Does this file include any ionospheric information?**
 <font color='red'>**RTA**</font>→ The GLONASS Navigation Message does not broadcast ionospheric
corrections.

 <font color='red'>**[3]**</font> **Which correction has to be applied to correct GLONASS system time
to UTC (SU timezone)?**
→ $Tutc = Tsv + TauN − GammaN ∗ (Tsv − Tb) + TauC$

$-1.117587089540e-08$

In [270]:
!head -29 $glofile

 <font color='red'>**[3]**</font> **Each of the other lines of GLONASS Navigation files contain 3
records with satellite position, velocity and Sun-Moon acceleration.
In which units are given? What information gives the fourth record
of each line?**
→ Units are: 
$km, 
km/s,   
km/s^2$
.
Message Frame Time, Satellite Health, Frequency Number, Information
Age.

##### GLONASS broadcast ephemeris and clock message parameters. 

![image2](img/headerNavGlonass.png)

For that reason, the ephemeris is not like GPS NAV header, With this Header, I have to biuld the Ephemeris for Glonass.
[help](ftp://igscb.jpl.nasa.gov/igscb/data/format/glonass.txt)


for each data (Observation) is like this
![image](img/GlonassEphemerisMessage.png)[help](http://www.navipedia.net/index.php/GLONASS_Satellite_Coordinates_Computation)

 <font color='red'>**[4]**</font> ** In the third row of each ephemeris block frequency information
is found. Which frequency was used by each GLONASS satellite
present in the file? What happened to force the range in the frequency
number?**<br />
→ R3 used the 7th frequency slot, so:<br />
    *$1602 + 0.5625 ∗ (-7) = -1605.9375 MHz.$<br />
R11 used the 4th frequency slot, so:<br />
    *$1602 + 0.5625 ∗ (-4) = -1604.25 MHz.$<br />

![img1](img/blockFrequency.png)

**Note:** The frequency numbers k were originally envisaged to provide **24 channels**, with $k = (1,...,24)$, but according to guidelines of the International Electric Communicationa Union (MSE), all GLONASS satellites launched **after 2005** will use $k = (-7,...,+6)$. This change was introduced to avoid interference problems with radioastronomy frequency bands and satellite communication services. [more](http://gage14.upc.es/gLAB/HTML/GLONASS_Navigation_Rinex_v2.11.html)

 <font color='red'>**[4]**</font> **  In the last row of each ephemeris block the period for each ephemeris
block is stated. When was last updated the satellite?**<br />
→ The broadcasted information is zero $(0)$ days old.

![img1](img/daysOld.png)

#### Build Ephemeris for GPS ####
For GPS is neccesary, set or build the Ephemeris with Time and ID of Satelllite
like this:

In [240]:
navHeader, navData = gpstk.readRinex3Nav(navfile)
# setup ephemeris store to look for satellite positions (GPS)
bcestore = gpstk.GPSEphemerisStore()
c=0

for navDataObj in navData:
    c += 1
    ephem = navDataObj.toGPSEphemeris()
    bcestore.addEphemeris(ephem)
    if(c==3):
        print bcestore
bcestore.SearchNear()
print "conteo",c
navData.close()

Dump of GPSEphemerisStore (detail level=1):
 BCE table for all satellites has 3 entries; Time span is 2015/11/12 08:00:00 GPS to 2015/11/13 04:00:00 GPS
 Search method is User
Sat G01 has   1 entries; Time span is 2015/11/13 00:00:00 GPS to 2015/11/13 00:00:00 GPS
Sat G02 has   1 entries; Time span is 2015/11/12 08:00:00 GPS to 2015/11/12 08:00:00 GPS
Sat G03 has   1 entries; Time span is 2015/11/13 02:00:00 GPS to 2015/11/13 02:00:00 GPS
END Dump of GPSEphemerisStore (detail level=1)

conteo 216


##### RINEX Navigation Files for GLONASS 

As the GLONASS navigation message differs in contents from the GPS message
too much, a special **GLONASS** navigation message file format has been defined.

The header section and the first data record (**epoch, satellite clock
information**) is similar to the GPS navigation file. The following records
contain the satellite position, velocity and acceleration, the clock and
frequency biases as well as auxiliary information as health, satellite
frequency (channel), age of the information.
[more](https://igscb.jpl.nasa.gov/igscb/data/format/rinex211.txt)

 <font color='blue'>The Navigator File of Glonass, doesn´t have the time and **ID of satellite** for set the **Ephemeris**, the Navigator have this for default</font>

In [283]:
navHeader, navData = gpstk.readRinex3Nav(glofile)
# setup ephemeris store to look for satellite positions (GLONASS)

bcestore2 = gpstk.GloEphemerisStore()
c=0
for navDataObj in navData:
    c += 1
    bcestore2.addEphemeris(navDataObj)
    if(c==2):
        print "bceStore",bcestore2
#bcestore2.SearchNear() 
print "conteo",c
navData.close()

bceStore Dump of GloEphemerisStore:
 Span is 2015/11/13 00:15:00 GLO to Begin_time with 2 entries; checkHealthFlag is F
Dump every record:
week   sow      = year/mn/dy hr:mi:sc Sys Sat   X                   Y                   Z                   VX                  VY                  VZ                  AX                  AY                  AZ                  TauN                GammaN            MFtime Hlth fNo AgeInfo
2015/11/13 00:15:00 GLO R02  2.303897949220e+03 -1.343449902340e+04  2.158463671880e+04  2.476346015930e+00 -1.601141929630e+00 -1.265949249270e+00  1.862645149230e-09  9.313225746150e-10 -9.313225746150e-10  1.615947112440e-04  1.818989403550e-12 442800   0  -4 0.00e+00
2015/11/13 23:45:00 GLO R10  8.809825683590e+03  9.732373535160e+03  2.184218554690e+04 -2.635574340820e-01  2.989327430730e+00 -1.232209205630e+00  3.725290298460e-09  0.000000000000e+00  0.000000000000e+00  7.189810276030e-07  0.000000000000e+00 441000   0  -7 0.00e+00
  End of GloEphemerisStore 

##### GLONASS broadcast ephemeris and clock message parameters. 

![image2](img/headerNavGlonass.png)

For that reason, the ephemeris is not like GPS NAV header, With this Header, I have to biuld the Ephemeris for Glonass.
[help](ftp://igscb.jpl.nasa.gov/igscb/data/format/glonass.txt)


Broadcast GLONASS ephemerides are contained in the RINEX navigation file for  **Satellite ID 4**. Geocentric <font color='blue'>positions (XYZ)</font> are highlighted in blue (1st X, 2nd Y and 3th Z), the <font color='green'>velocity</font> in green and the <font color='yellow'>[[[[</font>accelerations in yellow. The <font color='red'>clock parameters</font> are in red.

![img1](img/NavPositions.png)

<font color='red'>Red left</font> => Description:
Space Vehicle clock bias: **-TauN **

TUTC = TSV + TauN - GammaN*(Tsv-Tb) + TauC
Where TSV : Space Vehicle Time

<font color='red'>Red right</font> => Description:
Space Vehicle Relative Frequency Bias: **+GammaN **</br>

TUTC = TSV + TauN - GammaN*(Tsv-Tb) + TauC
Where TSV : Space Vehicle Time


### Satellite position is estimated using the 4th order Runge-Kutta numerical integration of    [here](http://gaussgge.unb.ca/GLONASS.ICD.pdf)

*Broadcast ephemerides have about 1 meter accuracy, but more precision may be necessary for geodetic application. Precise ephemerides can be used as an alternative to broadcast ephemerides. These ephemerides are calculated “a posteriori”, and they have a centimetric level accuracy. The SP3 format [here](http://igscb.jpl.nasa.gov/igscb/data/format/sp3_docu.txt) is used to describe these orbits and positions and satellite clock offsets are contained both for GPS and GLONASS constellations, with a rate of 15 minutes*.

Re-calculation of ephemeris from instant te $t_o$ instant $t_i$ within the interval of
measurement $(|τ_i| = |t_i - t_e| < 15 minutes)$ is performed using technique of
numerical integration of differential equations that describe motion of the satellites.
Right-hand parts of these equations take into account the accelerations determined by
gravitational constant $μ$ and second zonal coefficient $C20$, (that characterizes polar
flattening of Earth), and accelerations due to lunar-solar gravitational perturbation.
The equations are integrated in direct absolute geocentric coordinate system
$OX_aY_aZ_a$, connected with current equator and vernal equinox, using $4th$ order RungeKutta
technique as indicated below:

![Eq1](img/Equation1.png)

![Eq1](img/Equation2.png)

**1. Coordinates transformation to an inertial reference frame:**

The initial conditions ![IC](img/initialConditions.png), as broadcast in the GLONASS navigation message, are in the ECEF Greenwich coordinate system $PZ-90$. Therefore, and previous to orbit integration, they must be transformed to an absolute (inertial) coordinate system using the following expressions

**Position**
![position](img/position.png)

**Velocity**
![position](img/velocity.png)

 ![acc](img/acceleration.png) the acceleration components broadcast in the navigation message are the projections of luni-solar accelerations to axes of the ECEF Greenwich coordinate system. Thence, these accelerations must be transformed to the inertial system by:

 ![acc](img/accelerationNewSystem.png)

 ![acc](img/where.png)

 ![values](img/values.png)

In [279]:
c = 299792458
we = 7.2921159e-5

In [281]:
bcestore2.getIntegrationStep()

1.0

In [287]:
#for this test, i take "manual" the information of the first observation
posX = 8.809825683590e03
posY = 9.732373535160e03
posZ = 2.184218554690e04
velX = -2.635574340820e-01
velY = 2.989327430730e00
velZ = -1.232209205630e00
accX = 3.7252902984603e-09
accY = 0.0
accZ = 0.0

0.0

In [270]:
!head -29 $glofile

In [284]:
def rKN(x, fx, n, hs):
    k1 = []
    k2 = []
    k3 = []
    k4 = []
    xk = []
    for i in range(n):
        k1.append(fx[i](x)*hs)
    for i in range(n):
        xk.append(x[i] + k1[i]*0.5)
    for i in range(n):
        k2.append(fx[i](xk)*hs)
    for i in range(n):
        xk[i] = x[i] + k2[i]*0.5
    for i in range(n):
        k3.append(fx[i](xk)*hs)
    for i in range(n):
        xk[i] = x[i] + k3[i]
    for i in range(n):
        k4.append(fx[i](xk)*hs)
    for i in range(n):
        x[i] = x[i] + (k1[i] + 2*(k2[i] + k3[i]) + k4[i])/6
    return x

#### Version of the RINEX is 2.11 of 2015
is a Mixed Observation Data, have GPS and GLONASS

In [278]:
AllSat = []
Elevations = []
Azimuths = []
prefix = ""

obsHeader, obsData = gpstk.readRinex3Obs(obsfile)
obsObject = obsData.next()
#for obsObject in obsData: #round the data
for satID, datumList in obsObject.obs.iteritems():
       
    if(satID.system == satID.systemGPS):
        prefix = "GPS:"
        eph = bcestore.findEphemeris(satID, obsObject.time)
    elif(satID.system == satID.systemGlonass):
        prefix = "GLONASS:"
        #eph = bcestore2.findEphemeris(satID,obsObject.time)
        prueba = obsObject.time
        print "TRY", prueba
        print bcestore2.getXvt(0,prueba)
    else:
        prefix = str(satID.system)
    print prefix, satID.id

    svXvt = eph.svXvt(obsObject.time)
    print svXvt.getPos()
    elev = obsHeader.antennaPosition.elvAngle(svXvt.getPos())
    azim    = obsHeader.antennaPosition.azAngle(svXvt.getPos())
   
    AllSat.append(prefix + str(satID.id))
    Elevations.append(elev)
    Azimuths.append(azim)

GPS: 1
(-15459438.871682344, 848845.5130286308, 21525648.7011229)
GPS: 3
(-20972005.47795871, 9226510.918731103, 13452312.256385524)
GPS: 6
(6408811.944808661, 25632068.242831524, 2762650.286913051)
GPS: 11
(-19733752.84292065, -2840805.618219329, 17029360.975751553)
GPS: 17
(622814.4180263646, 15503495.90637051, 21891147.167720556)
GPS: 19
(-17475756.053354952, 7735852.528869855, 18424672.541175116)
GPS: 24
(16412879.253982931, 2100481.0732411677, 20773500.03237231)
GPS: 28
(-9293813.156037228, 21246450.60317787, 13507474.036448075)
GPS: 32
(-17262291.105189156, -3181242.7836655676, 19710335.444408286)
TRY 2457340 00000000 0.000000000000000 GPS


TypeError: in method 'GloEphemerisStore_getXvt', argument 2 of type 'gpstk::SatID const &'